In [2]:
# !pip install arxiv

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 408.7 kB/s eta 0:00:00 kB/s eta 0:00:01
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=3ba04fb150b7e7f9d42cdf77de0b70fb963ac2a14305956b7ebb6cb873fbe300
  Stored in directory: /home/dataviator/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
  Attempting uninstall: requests
    Found existing installation: requests 2.32.1
    Uninstalling requests-2.32.1:
      Successfully uninstalled requests-2.32.1


In [1]:
### Creating a wrapper around wiki, arxiv
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)


In [2]:
wiki_tool.name

'wikipedia'

In [3]:
from langchain.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(web_path="https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=250).split_documents(docs)

vector_db = FAISS.from_documents(documents,OpenAIEmbeddings())
retriver = vector_db.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x755305f5efd0>)

In [4]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool=create_retriever_tool(retriver, "langsmith_search","Seach for information about Langsmith, for any information about langsmith you must use this tool..")


In [5]:
retriver_tool.name

'langsmith_search'

In [6]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

arxiv.name

'arxiv'

In [7]:
tools = [wiki_tool, arxiv, retriver_tool]

In [10]:
from dotenv import load_dotenv

load_dotenv()
import os

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-3.5-turbo")


In [12]:
# Two ways of creating prompts
## 1. Aready checks - using ChatPromptTemplate
## 2. Using Hub

from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
### Querying from the tools using Agents
# from langchain.agents import create_openapi_tools_agent
# agent = create_openapi_tools_agent(llm, tools,prompt)

from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [16]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06

In [17]:
## Run the Agent
agent_executor.invoke({"input": "What is the purpose of Langsmith?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'purpose of Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first tr

{'input': 'What is the purpose of Langsmith?',
 'output': 'The purpose of LangSmith is to provide a platform for building production-grade Large Language Model (LLM) applications. It allows users to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. LangSmith works independently without the need for LangChain. Users can install LangSmith, create an API key, set up their environment, and log traces for evaluation. It offers multiple ways to log traces and evaluate applications effectively.'}

In [19]:
## Run the Agent
agent_executor.invoke({"input": "Tell me about GenAI"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `GenAI`


Page: Generative artificial intelligence
Summary: Generative artificial intelligence (generative AI, GenAI, or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models, often in response to prompts. Generative AI models learn the patterns and Generative artificial intelligence (generative AI, GenAI, or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models, often in response to prompts. GenAI models learn patterns and structures from data to create new content.

> Finished chain.


{'input': 'Tell me about GenAI',
 'output': 'Generative artificial intelligence (generative AI, GenAI, or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models, often in response to prompts. GenAI models learn patterns and structures from data to create new content.'}

In [21]:
## Run the Agent
agent_executor.invoke({"input": "Give me abstract of this paper at 2405.12751 arxiv, along with its main contribution "})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '2405.12751'}`


Published: 2024-05-21
Title: A Stealthy Backdoor Attack for Without-Label-Sharing Split Learning
Authors: Yuwen Pu, Zhuoyuan Ding, Jiahao Chen, Chunyi Zhou, Qingming Li, Chunqiang Hu, Shouling Ji
Summary: As a novel privacy-preserving paradigm aimed at reducing client computational
costs and achieviThe paper with arXiv ID 2405.12751 is titled "A Stealthy Backdoor Attack for Without-Label-Sharing Split Learning" by Yuwen Pu, Zhuoyuan Ding, Jiahao Chen, Chunyi Zhou, Qingming Li, Chunqiang Hu, and Shouling Ji. The main contribution of this paper is a novel privacy-preserving paradigm that aims to reduce client computational costs and achieve a stealthy backdoor attack for split learning without label sharing.

> Finished chain.


{'input': 'Give me abstract of this paper at 2405.12751 arxiv, along with its main contribution ',
 'output': 'The paper with arXiv ID 2405.12751 is titled "A Stealthy Backdoor Attack for Without-Label-Sharing Split Learning" by Yuwen Pu, Zhuoyuan Ding, Jiahao Chen, Chunyi Zhou, Qingming Li, Chunqiang Hu, and Shouling Ji. The main contribution of this paper is a novel privacy-preserving paradigm that aims to reduce client computational costs and achieve a stealthy backdoor attack for split learning without label sharing.'}

some helpful links for error
1. [Agent Quick start](https://python.langchain.com/v0.1/docs/modules/agents/quick_start/)
2. [Tools](https://python.langchain.com/v0.1/docs/modules/tools/)